In [ ]:
# !pip install tokenizers

In [1]:
!pip install gensim

  Using cached gensim-4.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.4 kB)
Using cached gensim-4.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.5 MB)


In [2]:
# Successfully installed huggingface-hub-0.23.3 tokenizers-0.19.1
# Successfully installed gensim-4.3.2

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tokenizers import Tokenizer, models, pre_tokenizers, trainers, normalizers
import random
import os

max_length = 3000

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHED'] = str(SEED)

from tensorflow.keras import mixed_precision as mixed_precision # Added 24052024
from tensorflow.keras.regularizers import l2


# policy = mixed_precision.Policy('mixed_float16') # Added 24052024
# mixed_precision.set_global_policy(policy) # 24052024

print(tf.__version__)

2024-06-14 12:23:06.156958: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.12.0


In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  2


2024-06-14 12:23:54.340188: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-14 12:23:54.340543: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-14 12:23:54.804252: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
from datetime import datetime

start = datetime.now()
print(start)

df_train = pd.read_csv('train.csv', delimiter =",", encoding='utf-8', encoding_errors=None)
df_train.head()

2024-06-14 12:24:21.352103


,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


In [5]:
len(df_train)

17307

In [6]:
df_aug = df_train.copy(deep=True)
df_aug['full_text'] = np.NaN

In [7]:
print(df_aug)

      essay_id  full_text  score
0      000d118        NaN      3
1      000fe60        NaN      3
2      001ab80        NaN      4
3      001bdc0        NaN      4
4      002ba53        NaN      3
...        ...        ...    ...
17302  ffd378d        NaN      2
17303  ffddf1f        NaN      4
17304  fff016d        NaN      2
17305  fffb49b        NaN      1
17306  fffed3e        NaN      2

[17307 rows x 3 columns]


In [ ]:
# from datetime import datetime

# def load_glove_vectors(glove_file):
#     with open(glove_file, 'r', encoding='utf-8') as f:
#         word_vectors = {}
#         for line in f:
#             parts = line.split()
#             word = parts[0]
#             vectors = np.array(parts[1:], dtype=np.float32)
#             word_vectors[word] = vectors
#     return word_vectors

# glove_file = 'glove.6B.300d.txt'
# print("Loading word vectors\n")
# print(datetime.now())
# word_vectors = load_glove_vectors(glove_file)
# print("Word vectors loaded\n")
# print(datetime.now())

In [ ]:
# from datetime import datetime
# from sklearn.metrics.pairwise import cosine_similarity

# def precompute_similar_words(word_vectors, batch_size=1000):
#     words = list(word_vectors.keys())
#     vectors = np.array(list(word_vectors.values()))      
    
#     similar_words_dict = {}
#     num_words = len(words)
    
#     for start in range(0, num_words, batch_size):
#         end = min(start + batch_size, num_words)
#         batch_vectors = vectors[start:end]
        
#         similarities = cosine_similarity(batch_vectors, vectors)
        
#         for i in range(end - start):
#             word_idx = start + i    
#             similar_idx = np.argmax(similarities[i])
#             similar_word = words[similar_idx]
#             similar_words_dict[words[word_idx]] = similar_word
            
#     return similar_words_dict

# print("Precomputing similar words\n")
# pre_start = datetime.now()
# print(pre_start)

# similar_words_dict = precompute_similar_words(word_vectors, batch_size=1000)

# pre_end = datetime.now()
# print(pre_end)
# print("Precompute complete\n")

In [8]:
from gensim.models import KeyedVectors
from datetime import datetime


glove_file = 'glove.6B.300d.txt'

print("Loading word vectors")
load_start = datetime.now()

word_vectors = KeyedVectors.load_word2vec_format(fname=glove_file, binary=False, unicode_errors='ignore', no_header=True, limit=400000)

load_end = datetime.now()
print(f"Word vectors loaded in {load_end - load_start}\n")

Loading word vectors
Word vectors loaded in 0:02:02.335329



In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def compute_most_similar(word):
    return word, word_vectors.most_similar(word, topn=1)[0][0]

print("Precomputing similar words\n")
comp_start = datetime.now()

similar_words_dict = {}
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(compute_most_similar, word): word for word in word_vectors.index_to_key}
    for future in as_completed(futures):
        word, most_similar = future.result()
        similar_words_dict[word] = most_similar

comp_end = datetime.now()
print(f"Precompute complete in {comp_end - comp_start}")

Precomputing similar words

Precompute complete in 1:09:06.516118


In [ ]:
import pickle

with open('similar_words_dict.pkl', 'wb') as fp:
    pickle.dump(similar_words_dict, fp)
    print('Dictionary saved successfully to file')    

Dictionary saved successfully to file


In [ ]:
with open('similar_words_dict.pkl', 'rb') as fp:
    similar_words_dict = pickle.load(fp)
    print('similar_words_dict loaded')

similar_words_dict loaded


In [ ]:
from multiprocessing import Pool

def augment_text_with_glove(text):
    augmented_text = []
    for word in text.split():
        if word in similar_words_dict:
            augmented_text.append(similar_words_dict[word])
        else:
            augmented_text.append(word)
    return ' '.join(augmented_text)

def parallel_augment_texts(texts, num_workers=8):
    with Pool(num_workers) as pool:
        augmented_texts = list(pool.imap(augment_text_with_glove, texts))
    return augmented_texts

print("Starting data augmentation now\n")
aug_start = datetime.now()
print("Augmentation start time is: ", aug_start)

df_aug['full_text'] = parallel_augment_texts(df_train['full_text'].tolist())

aug_end = datetime.now()
print("\nAugmentation end time is: ", aug_end)
print("\nData augmentation complete")

Starting data augmentation now

Augmentation start time is:  2024-06-14 13:36:44.545115

Augmentation end time is:  2024-06-14 13:36:47.263221

Data augmentation complete


In [ ]:
df_train_fin = pd.concat([df_train, df_aug], axis=0)
len(df_train_fin)

34614

In [ ]:
df_train_fin['score'] = df_train_fin['score'] - 1

In [ ]:
df_train_fin['full_text'].map(len).mean()

2180.515253943491

In [ ]:
# df_train_fin

In [ ]:
# !pip install --user tensorflow transformers

In [ ]:
# !pip install sentencepiece

In [ ]:
# from transformers import TFPegasusForConditionalGeneration, PegasusTokenizer

# model_name = "google/pegasus-xsum"
# tokenizer = PegasusTokenizer.from_pretrained(model_name)
# model = TFPegasusForConditionalGeneration.from_pretrained(model_name)

In [ ]:
# def paraphrase_text(text, num_return_sequences=1, num_beams=5):
#     inputs = tokenizer(text, return_tensors="tf", truncation=True, padding="longest")
    
#     outputs = model.generate(**inputs, num_beams=num_beams, num_return_sequences=num_return_sequences)
    
#     paraphrases = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#     return paraphrases
    

# # Example usage
# original_text = "This is an example sentence to be paraphrased."
# paraphrases = paraphrase_text(original_text, num_return_sequences=3)
# for i, para in enumerate(paraphrases):
#     print(f"Paraphrase {i + 1}: {para}")

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit as sss

splits = sss(n_splits=1, test_size=0.1, random_state=SEED)

for train_index, test_index in splits.split(df_train_fin, df_train_fin['score']):
    train_set = df_train_fin.iloc[train_index]
    test_set = df_train_fin.iloc[test_index]

print("Train set indices:", train_set.index)
print("Test set indices:", test_set.index)

Train set indices: Index([10133,  8163, 12374,  5693,  1891, 15415, 17269, 10705,  1580, 17231,
       ...
        4430,  4303,  5813,  1855,  8289, 12324,  8709,   900,  8848,  1733],
      dtype='int64', length=31152)
Test set indices: Index([ 5250, 15661,  3954,   942, 14459,  1641,  3924,  4980,  7771,  3207,
       ...
        5902, 14894, 11368,  4093,  2933,   175, 14873, 11650, 11625, 11198],
      dtype='int64', length=3462)


In [ ]:
len(train_set)

31152

In [ ]:
texts = train_set['full_text'].tolist()
labels = np.array(train_set['score'])

In [ ]:
!pip install --user spacy

In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 40.9 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import spacy
# Load the English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

# Function to apply lemmatization
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

from concurrent.futures import ProcessPoolExecutor

def process_chunk(chunk):
    return chunk.apply(lemmatize_text)

def parallel_lemmatize(data, num_processes):
    chunks = np.array_split(data, num_processes)
    with ProcessPoolExecutor(max_workers=num_processes) as executor:
        results = list(executor.map(process_chunk, chunks))
    return pd.concat(results, ignore_index=True)

print("\nApplying lemmatization now")
lem_start = datetime.now()
print("\nLemmatization start time is: ", lem_start)

num_processes = 8
train_set['full_text'] = parallel_lemmatize(train_set['full_text'], num_processes)

lem_end = datetime.now()
print("\nLemmatization end time is: ", lem_end)
print("\nData lemmatization complete")


Applying lemmatization now

Lemmatization start time is:  2024-06-14 13:37:07.543620

Lemmatization end time is:  2024-06-14 13:41:44.256199

Data lemmatization complete


/tmp/ipykernel_1/1993557821.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['full_text'] = parallel_lemmatize(train_set['full_text'], num_processes)


In [22]:
val_texts = test_set['full_text'].tolist()
val_labels = np.array(test_set['score'])

In [23]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=400000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=3000, padding='post')

In [24]:
val_sequences = tokenizer.texts_to_sequences(val_texts)
val_padded_sequences = pad_sequences(val_sequences, maxlen=3000, padding='post')

In [ ]:
# from tokenizers import Tokenizer, models, pre_tokenizers, trainers, normalizers
# # Initialize and train a BPE tokenizer
# tokenizer = Tokenizer(models.BPE())
# tokenizer.normalizer = normalizers.Sequence([normalizers.NFKC(), normalizers.Lowercase()])
# tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
# trainer = trainers.BpeTrainer(vocab_size=400000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]) # vocab size increased 30/05/2024
# tokenizer.train_from_iterator(texts, trainer)
# tokenizer.save("tokenizer.json")

In [ ]:
# Load the trained tokenizer and tokenize the texts
# tokenizer = Tokenizer.from_file("tokenizer.json")

In [ ]:
# tokenized_texts = [tokenizer.encode(text).ids for text in texts]

In [ ]:
# val_tokenized_texts = [tokenizer.encode(val_text).ids for val_text in val_texts]

In [ ]:
# # Pad the tokenized texts
# max_length = 3000
# padded_texts = pad_sequences(tokenized_texts, maxlen=max_length, padding='post')

# val_padded_texts = pad_sequences(val_tokenized_texts, maxlen=max_length, padding='post')

In [25]:
!pip install imblearn

  Using cached imblearn-0.0-py2.py3-none-any.whl.metadata (355 bytes)
  Using cached imbalanced_learn-0.12.3-py3-none-any.whl.metadata (8.3 kB)
Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)
Using cached imbalanced_learn-0.12.3-py3-none-any.whl (258 kB)


In [26]:
# Successfully installed imbalanced-learn-0.12.3 imblearn-0.0
from imblearn.over_sampling import SMOTE
# Apply SMOTE here
smote = SMOTE(random_state=SEED)
padded_sequences_res, labels_res = smote.fit_resample(padded_sequences, labels)

In [27]:
print("Original class distribution:\n", pd.Series(labels).value_counts())
print("Resampled class distribution:\n", pd.Series(labels_res).value_counts())

Original class distribution:
 2    11304
1     8501
3     7067
0     2253
4     1746
5      281
Name: count, dtype: int64
Resampled class distribution:
 1    11304
4    11304
3    11304
2    11304
0    11304
5    11304
Name: count, dtype: int64


In [33]:
# Create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((padded_sequences_res, labels_res))
dataset = dataset.shuffle(len(texts), seed=SEED) # increased 28/05/2024

val_dataset = tf.data.Dataset.from_tensor_slices((val_padded_sequences, val_labels))
val_dataset = val_dataset.batch(256) # increased 28/05/2024

In [29]:
@tf.keras.utils.register_keras_serializable()
class Attention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], input_shape[-1]), initializer="glorot_uniform", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(input_shape[-1],), initializer="zeros", trainable=True)
        self.u = self.add_weight(name="att_u", shape=(input_shape[-1],), initializer="glorot_uniform", trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x):
        # Compute the attention scores
        e = tf.keras.backend.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
        e = tf.tensordot(e, self.u, axes=1)
        a = tf.keras.backend.softmax(e, axis=1)

        # Compute the context vector
        a = tf.keras.backend.expand_dims(a, axis=-1)
        context = x * a
        context = tf.keras.backend.sum(context, axis=1)
        return context

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]
    
    def get_config(self):
        config = super(Attention, self).get_config()
        return config

In [30]:
tf.keras.backend.clear_session()

model = tf.keras.Sequential([
tf.keras.layers.Input(shape=(max_length,)),
tf.keras.layers.Embedding(input_dim=400000, output_dim=128),
tf.keras.layers.Conv1D(128, 5, activation='relu'),
tf.keras.layers.BatchNormalization(),       
tf.keras.layers.Dropout(0.4),
tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
tf.keras.layers.Dropout(0.4),
tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(0.01)),   
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True, kernel_regularizer=l2(0.01))),
tf.keras.layers.LSTM(32, return_sequences=True, kernel_regularizer=l2(0.01)),
Attention(),
tf.keras.layers.Dense(6, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

2024-06-14 13:42:22.624886: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-14 13:42:22.627266: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-14 13:42:22.628862: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3000, 128)         51200000  
                                                                 
 conv1d (Conv1D)             (None, 2996, 128)         82048     
                                                                 
 batch_normalization (BatchN  (None, 2996, 128)        512       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 2996, 128)         0         
                                                                 
 dense (Dense)               (None, 2996, 64)          8256      
                                                                 
 dropout_1 (Dropout)         (None, 2996, 64)          0         
                                                        

In [ ]:
# tf.keras.backend.clear_session()
# history = model.fit(dataset, epochs=50, validation_data=val_dataset)

In [31]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def create_adversarial_pattern(input_data, input_label, model):
    with tf.GradientTape() as tape:
        tape.watch(input_data)
        prediction = model(input_data, training=False)
        loss = loss_object(input_label, prediction)
    gradient = tape.gradient(loss, input_data)
    if gradient is None:
        raise ValueError("Gradient is None. Ensure that the input_data is being tracked by tf.GradientTape.")
    signed_grad = tf.sign(gradient)
    return signed_grad

In [ ]:
initial_epochs = 5
total_epochs = 10
epsilon = 0.1
batch_size = 256

train_dataset = dataset.batch(batch_size)

# Normal training phase
for epoch in range(initial_epochs):
    print(f'Initial Training Epoch {epoch + 1}/{initial_epochs}')
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_object(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        print(f"Step {step}, Loss: {loss_value.numpy()}")
        
# Adversarial training phase
for epoch in range(initial_epochs, total_epochs):
    print(f'Adversarial Training Epoch {epoch + 1 - initial_epochs}/{total_epochs - initial_epochs}')
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training = True)
            loss_value = loss_object(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        x_batch_train_float = tf.cast(x_batch_train, tf.float32)
        try:
            perturbations = create_adversarial_pattern(x_batch_train_float, y_batch_train, model)
        except ValueError as e:
            print(f"Error creating adversarial pattern: {e}")
            continue
        adversarial_examples = x_batch_train_float + epsilon * perturbations
        
        # Train on adversarial examples
        with tf.gradientTape() as tape:
            logits_adv = model(adversarial_examples, training=True)
            loss_adv_value = loss_object(y_batch_train, logits_adv)
        grads_adv = tape.gradient(loss_adv_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads_adv, model.trainable_weights))
        print(f"Step {step}, Adversarial Loss: {loss_adv_value.numpy()}")

print("Training completed.")

Initial Training Epoch 1/5


2024-06-14 13:43:17.376321: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [67824]
	 [[{{node Placeholder/_1}}]]
2024-06-14 13:43:17.376624: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [67824]
	 [[{{node Placeholder/_1}}]]
2024-06-14 13:43:17.824014: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8906
/opt/conda/lib/python3.10/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax ac

Step 0, Loss: 1.7953788042068481
Step 1, Loss: 1.7392375469207764
Step 2, Loss: 1.6893868446350098
Step 3, Loss: 1.6097919940948486
Step 4, Loss: 1.5961799621582031
Step 5, Loss: 1.5562680959701538
Step 6, Loss: 1.6016168594360352
Step 7, Loss: 1.5240468978881836
Step 8, Loss: 1.4971585273742676
Step 9, Loss: 1.5886601209640503
Step 10, Loss: 1.52681565284729
Step 11, Loss: 1.496572732925415
Step 12, Loss: 1.5226819515228271
Step 13, Loss: 1.528181552886963
Step 14, Loss: 1.4752845764160156
Step 15, Loss: 1.4686088562011719
Step 16, Loss: 1.436856985092163
Step 17, Loss: 1.5142769813537598
Step 18, Loss: 1.4411219358444214
Step 19, Loss: 1.4656282663345337
Step 20, Loss: 1.4602432250976562
Step 21, Loss: 1.4504635334014893
Step 22, Loss: 1.4341117143630981
Step 23, Loss: 1.457059383392334
Step 24, Loss: 1.4481346607208252
Step 25, Loss: 1.4163838624954224
Step 26, Loss: 1.430938482284546
Step 27, Loss: 1.4215164184570312
Step 28, Loss: 1.4217193126678467
Step 29, Loss: 1.41431403160095

In [ ]:
# optimizer = tf.keras.optimizers.Adam()
# loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# def create_adversarial_pattern(input_data, input_label, model):
    
#     input_data = tf.cast(input_data, tf.float32)
#     input_data = tf.Variable(input_data)
    
#     with tf.GradientTape() as tape:
#         tape.watch(input_data)
#         tape.watch(input_label)
#         prediction = model(input_data)
#         loss = loss_object(input_label, prediction)
    
#     print("Loss:", loss)
    
#     gradient = tape.gradient(loss, input_data)
    
#     if gradient is None:
#             raise ValueError("Gradient is None. Ensure that the input_data is being tracked by tf.GradientTape.")
    
    
#     signed_grad = tf.sign(gradient)
#     return signed_grad

# epochs = 10
# epsilon = 0.1


# dataset = dataset.batch(batch_size)

# for epoch in range(epochs):
#     print(f'Epoch {epoch + 1}/{epochs}')
#     for step, (x_batch_train, y_batch_train) in enumerate(dataset):
#         with tf.GradientTape() as tape:
#             logits = model(x_batch_train, training=True)
#             loss_value = loss_object(y_batch_train, logits)
        
#         grads = tape.gradient(loss_value, model.trainable_weights)
#         optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
#         perturbations = create_adversarial_pattern(x_batch_train, y_batch_train, model)
#         adversarial_examples = x_batch_train + epsilon * perturbations
        
        
#         with tf.GradientTape() as tape:
#             logits_adv = model(adversarial_examples, training=True)
#             loss_adv_value = loss_object(y_batch_train, logits_adv)
            
#         grads_adv = tape.gradient(loss_adv_value, model.trainable_weights)
#         optimizer.apply_gradients(zip(grads_adv, model.trainable_weights))
        
#     print(f'Loss at epoch {epoch + 1}: {loss_value.numpy().mean()}')

In [ ]:
# loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

# def create_adversarial_pattern(model, input_data, input_label):
        
#     input_data = tf.cast(input_data, tf.float32)  # Cast input data to float32
#     # input_data = tf.Variable(input_data)  # Convert to a variable
    
#     with tf.GradientTape() as tape:        
#         tape.watch(input_data)
#         prediction = model(input_data, training=True)
#         loss = loss_object(input_label, prediction)
    
#     gradient = tape.gradient(loss, input_data)
#     print(f"Input Data: {input_data.numpy()}")  # Debugging print
#     print(f"Prediction: {prediction.numpy()}")  # Debugging print
#     print(f"Loss: {loss.numpy()}")  # Debugging print
#     print(f"Gradient: {gradient}")  # Debugging print
#     if gradient is None:
#         raise ValueError("Gradient is None. Check if the input data is being watched correctly.")
#     signed_grad = tf.sign(gradient)
#     return signed_grad

In [ ]:
# def train_step(model, optimizer, loss_fn, x, y, epsilon=0.01):
    
#     x = tf.cast(x, tf.float32)  # Ensure x is of float32 type

#     with tf.GradientTape() as tape:
#         predictions = model(x, training=True)
#         loss = loss_fn(y, predictions)
    
#         gradients = tape.gradient(loss, model.trainable_variables)
#         optimizer.apply_gradients(zip(gradients, model.trainable_variables))

#     # Adversarial training
#     try:
#         perturbations = create_adversarial_pattern(model, x, y)
#         x_adv = x + epsilon * perturbations
#         x_adv = tf.clip_by_value(x_adv, 0, 1)

#         with tf.GradientTape() as tape:
#             predictions_adv = model(x_adv, training=True)
#             loss_adv = loss_fn(y, predictions_adv)

#         gradients_adv = tape.gradient(loss_adv, model.trainable_variables)
#         optimizer.apply_gradients(zip(gradients_adv, model.trainable_variables))

#     except ValueError as e:
#         print(f"Adversarial training error: {e}")
#         loss_adv = tf.constant(0.0)
    
#     return loss, loss_adv

In [ ]:
# def val_step(model, loss_fn, x, y):
    
#     # x = tf.cast(x, tf.float32)
#     # x = tf.Variable(x)
    
#     predictions = model(x, training=False)
#     loss = loss_fn(y, predictions)
#     return loss

In [ ]:
# # Define training parameters
# epochs = 5
# optimizer = tf.keras.optimizers.Adam()
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

# for epoch in range(epochs):
#     print(f"Epoch {epoch + 1}/{epochs}")
    
#     # Training loop
#     for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
#         print(f"Step {step}, x_batch_train shape: {x_batch_train.shape}, y_batch_train shape: {y_batch_train.shape}")
#         try:
#             loss, loss_adv = train_step(model, optimizer, loss_fn, x_batch_train, y_batch_train)
#             print(f"Training loss: {loss.numpy()}, Adversarial training loss: {loss_adv.numpy()}")
#         except ValueError as e:
#             print(e)
    
#     # Validation loop
#     for step, (x_batch_val, y_batch_val) in enumerate(val_dataset):
#         print(f"Validation Step {step}, x_batch_val shape: {x_batch_val.shape}, y_batch_val shape: {y_batch_val.shape}")
#         val_loss = val_step(model, loss_fn, x_batch_val, y_batch_val)
#         print(f"Validation loss: {val_loss.numpy()}")

In [ ]:
import matplotlib.pyplot as plt

# Plot utility
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

# Plot the accuracy and results 
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
# best_model = tf.keras.models.load_model('best_model.keras', custom_objects={'Attention':Attention})

In [35]:
df_test = pd.read_csv('test.csv')
df_test.head()

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


In [37]:
# tokenized_pred_texts = [tokenizer.encode(pred_text).ids for pred_text in pred_texts]
# padded_pred_texts = pad_sequences(tokenized_pred_texts, maxlen=max_length, padding='post')
# numpy_pred_texts = np.array(padded_pred_texts)

pred_texts = df_test['full_text'].tolist()
pred_sequences = tokenizer.texts_to_sequences(pred_texts)
padded_pred_sequences = pad_sequences(pred_sequences, maxlen=3000, padding='post')
numpy_pred_texts = np.array(padded_pred_sequences)

In [38]:
preds = model.predict(numpy_pred_texts)
preds

2024-06-14 14:13:56.656129: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-14 14:13:56.658381: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-14 14:13:56.660128: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 1s 1s/step


array([[1.05545274e-04, 1.64994244e-02, 7.47959316e-01, 2.28391692e-01,
        7.02911103e-03, 1.49089656e-05],
       [7.47881204e-05, 2.82575004e-02, 9.52823639e-01, 1.86553989e-02,
        1.86057718e-04, 2.61780337e-06],
       [1.21311932e-04, 1.06689206e-03, 5.30397799e-03, 2.04276174e-01,
        7.87877142e-01, 1.35456608e-03]], dtype=float32)

In [39]:
max_preds = np.argmax(preds, axis=1)
res_lst = max_preds + 1
res_lst

array([3, 3, 5])

In [ ]:
df_sub = pd.DataFrame()
df_sub['essay_id'] = df_test['essay_id']
df_sub

In [ ]:
df_sub['score'] = res_lst.transpose()
df_sub['score'] = df_sub['score'].astype('int')
df_sub

In [ ]:
df_sub.to_csv('submission.csv', header=True, index=False)

In [ ]:
import gc

gc.collect()

end = datetime.now()
print(end)